In [1]:
from webscraper import Scraper

In [2]:
scraper = Scraper() #call scraper class
scraper.accept_cookies()
scraper.navigate()
scraper.search_bar("final fantasy") #add search keyword here



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been 

Webpage loaded successfully
'Accept Cookies' button clicked
Search bar opened
Search keywords entered
Submit search button clicked - redirected to results


'final fantasy'

In [3]:
import selenium
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager #installs Chrome webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

container = scraper.find_container()
product_links = scraper.collect_page_links() #use product_list[] to check links

In [29]:
from selenium.common.exceptions import NoSuchElementException
import uuid #universally unique id
import time
from bs4 import BeautifulSoup
import requests
import re
import os, shutil, os.path

#add link and product info to dictionary
product_dict = {"Link": [], "Product Name" :[], "Price" : [], "Description" : [], "ID": [], "UUID":[]}
image_dict = {"Link": [], "Product UUID": [], "Image UUID": [], "Image": []}

def retrieve_data():
   #link_list #lists all urls in specified webpage section
   for link in product_links:
      #pass any links which relate to games
      if "download" in link:
         pass
      elif "remastered" in link:
         pass
      elif "pc" in link:
         pass
      elif "ps4" in link:
         pass
      else:
         scraper.driver.get(link)
         time.sleep(2)
         product_dict["Link"].append(link)

         try: #get product name
            product_name = scraper.driver.find_element(By.XPATH, 
               '//*[@id="desktop-wrapper-title"]/div/header/h1')
               #shortened form of //*[@id="responsive-wrapper-title"]/header/h1
               #.//h1[@class="product-title"] - works some of the time
               #//*[@id="desktop-wrapper-title"]/div/header/h1
               #//*[@id="desktop-wrapper-title"]/div/header/h1
            product_dict["Product Name"].append(product_name.text)
            print("Product name obtained")
         except NoSuchElementException: #not all links accessed will have the same attributes
            product_dict["Product Name"].append("N/A")
         
         time.sleep(3)
         try: #get price
            price = scraper.driver.find_element(By.XPATH, '//span[@class = "prices"]')
            #//*[@id="main-content"]/article/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/span/span
               #shortened form of //*[@id="main-content"]/article/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/span
            product_dict["Price"].append(price.text)
            print("Price obtained")
         except NoSuchElementException:
            product_dict["Price"].append("N/A")
         
         time.sleep(3)
         try: #get product description
            desc = scraper.driver.find_element(By.XPATH, './/div[@class="tab-pane-content"]')
               #shortened form of //*[@id="desc-collapse"]/div/div
            product_dict["Description"].append(desc.text)
            print("Product description obtained")
         except NoSuchElementException:
            product_dict["Description"].append("N/A")
         
         time.sleep(3)
         try: #get product SKU/ID from URL
            #to get SKU - product details tab needs to be clicked and then an if statement to get the SKU otherwise other info will be obtained
            r = link.rsplit("/", 6) #split link 6 times after every '/'
            product_dict["ID"].append(r[5])
            print("Product ID obtained")
         except NoSuchElementException:
            product_dict["ID"].append("N/A")

         time.sleep(3)
         try: #generate V4 UUID for product and image
            product_uuid = uuid.uuid4()
            image_uuid = uuid.uuid4()
            product_dict["UUID"].append(product_uuid), image_dict["Product UUID"].append(product_uuid)
            image_dict["Image UUID"].append(image_uuid)

            print("UUID generated")
         except NoSuchElementException:
            product_dict["UUID"].append("N/A")
            image_dict["Product UUID"].append("N/A")
            image_dict["Image UUID"].append("N/A")
         
         time.sleep(3)
         try: #download and save product image using product ID as file name
            image = f"{r[5]}.png"
            with open(image, "wb") as file: #wb = write and binary mode
               img = scraper.driver.find_element(By.XPATH, 
                  '//img[@class="boxshot lazyloaded"]')
               #//*[@id="main-content"]/article/div[1]/div/div[1]/div[2]/figure/img
               file.write(img.screenshot_as_png)
               time.sleep(3)
            image_dict["Image"].append(image)
            print("Image downloaded")
         except:
            image_dict["Image"].append("N/A")
            print("Image not downloaded")
         time.sleep(3)

         try:
            html_page = requests.get(link).content #send get request for each link in range and get content of each request
            soup = BeautifulSoup(html_page)
            images = []
            for img in soup.find_all("img"): #finds .jpg images only
                  
                  images.append(img.get("srcset"))

            #print(images[2])

               #add image link(s) to image_dict
            image_link = images[2]
            image_dict["Link"].append(image_link)
               #image_dict1["Link"].append(image_link1)
            print("Image link added")
         except:
            image_dict["Link"].append("N/A")
            print("Image link not added")
         time.sleep(3)
      
retrieve_data()

Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Image downloaded
Image link added
Product name obtained
Price obtained
Product descriptio

In [30]:
import pandas as pd

pd.DataFrame(product_dict) #displays product dictionary in panda dataframe
#pd.DataFrame(image_dict) #displays image dictionary in panda dataframe

,Link,Product Name,Price,Description,ID,UUID
0,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY MINI PLUSH: FINAL FANTASY IX ZIDANE,£15.99,Zidane of FINAL FANTASY IX is now available as...,462642,b048b3b0-9782-4bf2-87a7-d53a9110b1a9
1,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY JIGSAW PUZZLE - FINAL FANTASY VI...,£14.99,Square Enix is proud to offer this 500-piece j...,604828,0e000d07-fde9-49dd-9de1-f291cdd31bae
2,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY VII,from £9.99,Steam key issued upon purchase. Activate via y...,603952,63c38f3a-2e02-4b77-aa42-899c5fba67bd
3,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY MAT - CHOCOBO,£23.99,"Amazingly cute mats of Chocobo, Cactuar, and M...",598069,a177000c-aea3-4f9c-9c58-fda0d3ef50cf
4,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY PLUSH [CACTUAR],£22.99,A plush toy of Cactuar and one of the traditio...,306733,87a27c86-7dac-4a47-8308-5b709cdfbba9
5,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY JUMBO CACTUAR,£76.99,This jumbo plush with be a 1/1 scale as the Ca...,598598,e4686839-34e1-4880-ae02-8d8e79249a37
6,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY MAT - CACTUAR,£23.99,"Amazingly cute mats of Chocobo, Cactuar, and M...",598050,1653640e-709f-4f73-9887-fc7e7729c06f
7,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY PLUSH MASCOT - CACTUAR,£14.99,FINAL FANTASY mascot characters join the fray ...,560386,0e3ffd76-fbd1-4f0d-9fe4-2404855fc52f
8,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY STACKING MUG - CHOCOBO,£11.49,"Introducing the cute, new Stacking Mugs, featu...",572126,7e483aeb-aecd-4b15-9df5-7241e0d52eac
9,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY XIV POSTER COLLECTION,£22.99,"Celebrate the lush, magnificent artwork of the...",676788,5762f3f7-e0a5-46d7-a6d5-53ebc153f21b


In [31]:
import os, os.path
import json
from json import JSONEncoder
from uuid import UUID

new_folder = "../raw_data"
file = "data"
create_file = os.path.join(new_folder, file+".json") #add file type here

#Dealing with no UUID serialization support in json
JSONEncoder_olddefault = JSONEncoder.default
def JSONEncoder_newdefault(self, o):
    if isinstance(o, UUID): return str(o)
    return JSONEncoder_olddefault(self, o)
JSONEncoder.default = JSONEncoder_newdefault


try: #create raw_data folder in current directory - check if folder already exists
    if not os.path.exists(new_folder):
        os.mkdir(new_folder) #create folder if it doesn't exist
        with open(create_file, "w") as fp: #specify path here 
                json.dump(product_dict, fp,  indent=4)
                
    elif os.path.exists(new_folder): #if folder already exists
        with open(create_file, "w") as fp: 
                json.dump(product_dict, fp,  indent=4)
except FileExistsError:
    print("Already exists")

In [32]:
#move files into a new directory/folder
import os, shutil, os.path #shutil - to save it locally

new_folder1 = "images" #create images folder in current directory
sourcepath = "../webscraping"
source = os.listdir(sourcepath)
destinationpath = "../webscraping/images"

try:
 if not os.path.exists(new_folder1):
  os.mkdir(new_folder1)
  for files in source:
      if files.endswith('.png'):
          shutil.move(os.path.join(sourcepath, files), os.path.join(destinationpath, files))    
 elif os.path.exists(new_folder1):
      for files in source:
        if files.endswith('.png'):
            shutil.move(os.path.join(sourcepath, files), os.path.join(destinationpath, files)) 
except FileExistsError:
 print("Already exists")